# eCommerce Product Recommendation - Part 2

# 1. Load Raw Data

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
sns.set()

In [2]:
aisles = pd.read_csv(r'/Users/jichenxi/Documents/MGT_Group_Project/aisles.csv')
departments = pd.read_csv(r'/Users/jichenxi/Documents/MGT_Group_Project/departments.csv')
order_products_prior = pd.read_csv(r'/Users/jichenxi/Documents/MGT_Group_Project/order_products_prior.csv')
order_products_train = pd.read_csv(r'/Users/jichenxi/Documents/MGT_Group_Project/order_products_train.csv')
orders = pd.read_csv(r'/Users/jichenxi/Documents/MGT_Group_Project/orders.csv')
products = pd.read_csv(r'/Users/jichenxi/Documents/MGT_Group_Project/products.csv')
#all_data = pd.read_csv('all_data.csv')

#data description: https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b


In [116]:
print(aisles.shape)
print(departments.shape)
print(order_products_prior.shape)
print(order_products_train.shape)
print(orders.shape)
print(products.shape)

(134, 2)
(21, 2)
(793924, 4)
(53704, 4)
(82683, 7)
(49688, 4)


# Construct Model Label

cant't use `reordered`column data as model label




In [131]:
#merge order details with training data using order_id
#get each product id, associated order id and training details
train_details = order_products_train.merge(orders, on = 'order_id')

#get all unique training user_ids
train_user_ids = set(orders[orders['eval_set'] == 'train']['user_id'])

#contruct unique key for each user_id-product_id entry
train_unique_key = train_details['user_id'].astype('str') + '_' + train_details['product_id'].astype('str')

In [133]:
#train_details

In [134]:
train_unique_key.head()

0    173934_13176
1    173934_39922
2     173934_5258
3    173934_21137
4     129386_6046
dtype: object

In [135]:
#merge order details with prior order data using order_id
#get each product id, associated order id and prior order details
prior_details = order_products_prior.merge(orders, on = 'order_id')

In [136]:
#prior_details.head()

In [137]:
model_all_data = prior_details[prior_details.user_id.isin(train_user_ids)][['user_id','product_id','order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']]


In [138]:
#get all data, including training and prior
model_all_data = prior_details[prior_details.user_id.isin(train_user_ids)][['user_id','product_id','order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']]


#in model_all_data，should be only one user - product unique ntry
model_all_data = model_all_data.drop_duplicates(subset=['user_id', 'product_id'])
#construct unique key 
model_all_data['unique_key'] = model_all_data['user_id'].astype('str') + '_' + model_all_data['product_id'].astype('str')

In [139]:
model_all_data.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key
0,152610,30597,22,6,8,10.0,152610_30597
1,152610,15221,22,6,8,10.0,152610_15221
2,152610,43772,22,6,8,10.0,152610_43772
3,152610,37886,22,6,8,10.0,152610_37886
4,152610,37215,22,6,8,10.0,152610_37215


In [140]:
model_all_data.shape

(329806, 7)

In [141]:
#set all defalu label value to 0
model_all_data['label'] = 0

# get all training data entries and set label value to 1
model_all_data.loc[model_all_data.unique_key.isin(train_unique_key), 'label'] = 1

In [142]:
model_all_data.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key,label
0,152610,30597,22,6,8,10.0,152610_30597,0
1,152610,15221,22,6,8,10.0,152610_15221,0
2,152610,43772,22,6,8,10.0,152610_43772,0
3,152610,37886,22,6,8,10.0,152610_37886,0
4,152610,37215,22,6,8,10.0,152610_37215,0


In [143]:
model_all_data.shape

(329806, 8)

# Construct Model Features

### Feature Group 1: user-product activity features
* unique to each user-product pair

In [144]:
#construct df headers using feature names
user_product_features = ['user_product__total_orders',
                         'user_product__add_to_cart_order_mean',
                         'user_product__reordered_mean',
                         'user_product__most_dow',
                         'user_product__most_hod']

Construct features to each datapoint with aggregations.

For example, for each user-product datapoint: 
* count of all order_ids that this user had purchased this product. 
* the mean add_to_cart_order that this user has added this product to the cart
* the mean of if this product is reordered by this user
* the most ordered day of week that this user to this product
* the most ordered hour of day that this user to this product

In [145]:
# expect 2mins execution
df_user_product_features = (prior_details.groupby(['product_id','user_id'],as_index=False) 
                                           .agg(OrderedDict(
                                                   [('order_id','count'), 
                                                    ('add_to_cart_order','mean'), 
                                                    ('reordered', 'mean'), 
                                                    ('order_dow', (lambda x: x.mode()[0])), #dow that this user purchased most of this product
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), #hod that this user purchased most of this product
                                                    ])))
df_user_product_features.columns = ['product_id', 'user_id'] + user_product_features

In [146]:
df_user_product_features.head()

,product_id,user_id,user_product__total_orders,user_product__add_to_cart_order_mean,user_product__reordered_mean,user_product__most_dow,user_product__most_hod
0,1,21285,1,3.0,0.00,0,15
1,1,47549,4,4.0,0.75,2,14
2,1,54136,1,3.0,0.00,2,13
3,1,54240,1,2.0,0.00,2,16
4,1,95730,1,1.0,0.00,1,14


In [147]:
#merge features results with model data
model_all_data = model_all_data.merge(df_user_product_features, on = ['user_id', 'product_id'])

In [149]:
model_all_data.shape

(329806, 13)

### Feature Group 2: product features

**2 types of product features:**
* product features based on user behavior
* product own info as feature

In [150]:
#construct df headers using feature names
product_features = ['product__total_orders',
                     'product__add_to_cart_order_mean',
                     'product__total_users',
                     'product__reordered_mean',
                     'product__most_dow',
                     'product__most_hod',
                     'product__days_since_prior_order_mean'
                     ]

In [151]:
# construct features to each datapoint (with some aggregations)
df_product_features = (prior_details.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'), 
                                                    ('add_to_cart_order','mean'), 
                                                    ('user_id', 'nunique'), 
                                                    ('reordered', 'mean'), 
                                                    ('order_dow', (lambda x: x.mode()[0])),
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])),
                                                    ('days_since_prior_order', 'mean') 
                                                    ])))
df_product_features.columns = ['product_id'] + product_features

In [152]:
#merge all product features to model all data
model_all_data = model_all_data.merge(df_product_features, on = ['product_id'])

In [153]:
#merge with product aisle, department info
model_all_data = model_all_data.merge(products[['product_id','aisle_id', 'department_id']], 
                                      on = ['product_id'])

In [154]:
#rename col names
model_all_data.rename(columns={'aisle_id': 'product__aisle_id',\
                               'department_id': 'product__department_id'}, inplace=True)

**potentially: NLP on product name/aisle and tokenization, and then add to product feature list **

### Feature Group 3: user features
* user's activity across all products

In [155]:
#construct df headers
user_features = ['user__order_count',
                  'user__product_count',
                  'user__days_since_prior_order_mean',
                  'user__reordered_mean',
                  'user__most_dow',
                  'user__most_hod',
                  ]

In [156]:
# construct df of user features with aggregated values on user level 
df_user_features = (prior_details.groupby(['user_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'), 
                                                    ('product_id','count'), 
                                                    ('days_since_prior_order','mean'),
                                                    ('reordered', 'mean'), 
                                                    ('order_dow', (lambda x: x.mode()[0])), 
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])),
                                                    ])))
df_user_features.columns = ['user_id'] + user_features

In [157]:
#merge user feature info to model all data
model_all_data = model_all_data.merge(df_user_features, on = ['user_id'])

In [162]:
model_all_data.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key,label,user_product__total_orders,user_product__add_to_cart_order_mean,...,product__most_hod,product__days_since_prior_order_mean,product__aisle_id,product__department_id,user__order_count,user__product_count,user__days_since_prior_order_mean,user__reordered_mean,user__most_dow,user__most_hod
0,152610,30597,22,6,8,10.0,152610_30597,0,5,2.00,...,9,12.592593,53,16,26,408,10.953804,0.375,6,12
1,152610,15221,22,6,8,10.0,152610_15221,0,8,8.25,...,12,13.653846,84,16,26,408,10.953804,0.375,6,12
2,152610,43772,22,6,8,10.0,152610_43772,0,2,3.00,...,12,12.008565,83,4,26,408,10.953804,0.375,6,12
3,152610,37886,22,6,8,10.0,152610_37886,0,2,3.50,...,14,11.489362,53,16,26,408,10.953804,0.375,6,12
4,152610,37215,22,6,8,10.0,152610_37215,0,1,5.00,...,13,11.490066,98,7,26,408,10.953804,0.375,6,12


In [160]:
model_all_data['label'].value_counts()

0    297467
1     32339
Name: label, dtype: int64

In [161]:
model_all_data.shape

(329806, 28)

In [163]:
# Save the model_all_data dataframe as csv file to the file path we selected above.
model_all_data.to_csv('model_all_data.csv', index=False)